<a href="https://colab.research.google.com/github/AnanditaRai/ana/blob/main/fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

--2024-12-27 18:34:49--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.3’

book-crossings.zip. 100%[===================>]  24.88M  96.2MB/s    in 0.3s    

2024-12-27 18:34:49 (96.2 MB/s) - ‘book-crossings.zip.3’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
# add your code here - consider creating a new cell for each section of code
# Remove books with fewer than 100 ratings
book_ratings_count = df_ratings.groupby('isbn').size()
popular_books = book_ratings_count[book_ratings_count >= 100].index
df_ratings_filtered = df_ratings[df_ratings['isbn'].isin(popular_books)]

# Remove users with fewer than 200 ratings
user_ratings_count = df_ratings_filtered.groupby('user').size()
active_users = user_ratings_count[user_ratings_count >= 200].index
df_ratings_filtered = df_ratings_filtered[df_ratings_filtered['user'].isin(active_users)]

# Create the user-item matrix (user-item ratings matrix)
user_item_matrix = df_ratings_filtered.pivot(index='user', columns='isbn', values='rating').fillna(0)

# Convert the matrix to a sparse format for efficiency
user_item_matrix_sparse = csr_matrix(user_item_matrix.values)


In [ ]:
# function to return recommended books - this will be tested
# Initialize the Nearest Neighbors model
knn = NearestNeighbors(n_neighbors=6, metric='cosine', algorithm='brute')

# Fit the model to the user-item matrix
knn.fit(user_item_matrix_sparse)

def get_recommends(book=""):
    # Get the ISBN of the book from the title
    book_isbn = df_books[df_books['title'] == book]['isbn'].values[0]

    # Find the index of the book in the user-item matrix
    book_index = user_item_matrix.columns.get_loc(book_isbn)

    # Get the nearest neighbors for the book
    distances, indices = knn.kneighbors(user_item_matrix_sparse[:, book_index].reshape(1, -1))

    # Prepare the list of recommended books
    recommended_books = []
    for i in range(1, len(indices[0])):  # Start from 1 to skip the book itself
        similar_book_isbn = user_item_matrix.columns[indices[0][i]]
        similar_book_title = df_books[df_books['isbn'] == similar_book_isbn]['title'].values[0]
        distance = distances[0][i]
        recommended_books.append([similar_book_title, distance])

    return [book, recommended_books]


In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()